<a href="https://colab.research.google.com/github/dougnr89/Estudo_Roubo_Veiculos_SP/blob/main/TCC_Rascunho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Bibliotecas**

In [1]:
!pip install squarify -qqq

import pandas as pd
import numpy as np
import seaborn as sns
import squarify
import glob
import re
import itertools
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib
plt.style.use('fivethirtyeight')

import plotly.offline as py
py.init_notebook_mode(connected=True)

import plotly.graph_objs as go
import plotly.tools as tls
from folium import Map
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster

import warnings
warnings.filterwarnings("ignore")

**Dataset**

In [2]:
dir_path = "/content/"

def importaData(files):

    dados = dir_path + files
    allFiles = glob.glob('/content/*.csv')

    list_ = []
    cols = ['ANO_BO', 'DATAOCORRENCIA', 'PERIDOOCORRENCIA', 'BAIRRO', 'CIDADE', 'LATITUDE', 'LONGITUDE', 'LOGRADOURO', 'DESCRICAOLOCAL', 'DESCR_MARCA_VEICULO', 'DESCR_TIPO_VEICULO']

    for file_ in allFiles:
        df = pd.read_csv(file_,index_col=None, header=0, delimiter=';', usecols=cols, encoding='ISO-8859-1')
        list_.append(df)

    df = pd.concat(list_, axis = 0, ignore_index = True)

    return df

all_data = importaData('*.csv')

In [3]:
all_data.tail()

,ANO_BO,DATAOCORRENCIA,PERIDOOCORRENCIA,LOGRADOURO,BAIRRO,CIDADE,LATITUDE,LONGITUDE,DESCRICAOLOCAL,DESCR_MARCA_VEICULO,DESCR_TIPO_VEICULO
116594,2022,30/11/2022,A NOITE,Rodovia Presidente Tancredo de Almeida N,NaN,FRANCO DA ROCHA,"-23,2980694","-46,7839643",Via pública,HONDA/CG 160 TITAN,MOTOCICLO
116595,2022,30/11/2022,A NOITE,Rodovia Presidente Tancredo de Almeida N,NaN,FRANCO DA ROCHA,"-23,2980694","-46,7839643",Via pública,HONDA/CG 160 TITAN,MOTOCICLO
116596,2022,30/11/2022,A NOITE,Rua Padre Augusto Rizi,Jardim Ana Maria,S.ANDRE,"-23,6265492","-46,4875959",Garagem ou abrigo de residência,I/NISSAN KICKS SV CVT,AUTOMOVEL
116597,2022,29/11/2022,A TARDE,Avenida Italo Adami,Vila Zeferina,ITAQUAQUECETUBA,"-23,492251","-46,349015",Via pública,I/TOYOTA COROLLA XEI,AUTOMOVEL
116598,2022,30/11/2022,A NOITE,RUA DO OLEODUTO,MONTANHÃO,S.BERNARDO DO CAMPO,NaN,NaN,Via pública,CITROEN/C3 EXCL 16 FLEX,AUTOMOVEL


In [5]:
all_data.describe()

,ANO_BO
count,116599.000000
mean,2022.006244
std,0.080173
min,2021.000000
25%,2022.000000
50%,2022.000000
75%,2022.000000
max,2023.000000


In [6]:
print("Dados e Valores: ")

all_data.info()

Dados e Valores: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116599 entries, 0 to 116598
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ANO_BO               116599 non-null  int64 
 1   DATAOCORRENCIA       116599 non-null  object
 2   PERIDOOCORRENCIA     116599 non-null  object
 3   LOGRADOURO           110975 non-null  object
 4   BAIRRO               113449 non-null  object
 5   CIDADE               116521 non-null  object
 6   LATITUDE             107338 non-null  object
 7   LONGITUDE            107338 non-null  object
 8   DESCRICAOLOCAL       116599 non-null  object
 9   DESCR_MARCA_VEICULO  82346 non-null   object
 10  DESCR_TIPO_VEICULO   82219 non-null   object
dtypes: int64(1), object(10)
memory usage: 9.8+ MB


In [7]:
def remocaoduplicados(df):

    print('Quantidade duplicados: ', df.duplicated().sum())
    df.drop_duplicates(inplace=True)
    print('Quantidade duplicados após remoção: ', df.duplicated().sum())

    return df

all_data = remocaoduplicados(all_data)

Quantidade duplicados:  58248
Quantidade duplicados após remoção:  0


In [8]:
def filtroSP(df):

    df.CIDADE.fillna('S.PAULO')
    df = df[df['CIDADE'] == 'S.PAULO']
    del df['CIDADE']

    return df

all_data = filtroSP(all_data)

In [9]:
def lCase(df, cols):
    for col in cols:
        df[col] = df[col].str.title()

    return df

colsToLower = ["PERIDOOCORRENCIA", "LOGRADOURO", "BAIRRO", "DESCRICAOLOCAL", "DESCR_TIPO_VEICULO"]

all_data = lCase(all_data, colsToLower)

In [10]:
all_data.loc[(all_data['DESCR_TIPO_VEICULO'] == "MOTONETA") | (all_data['DESCR_TIPO_VEICULO'] == "MOTOCICLO") | (all_data['DESCR_TIPO_VEICULO'] == "CICLOMOTO"),'TIPO_VEICULO'] = "MOTO"
all_data.loc[(all_data['DESCR_TIPO_VEICULO'] == "CAMINHONETE") | (all_data['DESCR_TIPO_VEICULO'] == "CAMIONETA") | (all_data['DESCR_TIPO_VEICULO'] == "UTILIT?RIO") | (all_data['DESCR_TIPO_VEICULO'] == "AUTOMOVEL"),'TIPO_VEICULO'] = "AUTOMOVEL"
all_data.loc[(all_data['DESCR_TIPO_VEICULO'] == "CAMINH?O") | (all_data['DESCR_TIPO_VEICULO'] == "CAMINH?O TRATOR") | (all_data['DESCR_TIPO_VEICULO'] == "SEMI-REBOQUE") | (all_data['DESCR_TIPO_VEICULO'] == "REBOQUE"),'TIPO_VEICULO'] = "CAMINHAO"
all_data.loc[(all_data['DESCR_TIPO_VEICULO'] == "MICRO-ONIBUS") | (all_data['DESCR_TIPO_VEICULO'] == "ONIBUS"),'TIPO_VEICULO'] = "ONIBUS"
all_data.loc[(all_data['TIPO_VEICULO'].isnull()) & (all_data['DESCR_TIPO_VEICULO'].notnull()),'TIPO_VEICULO'] = "OUTROS"


In [11]:
all_data.loc[(all_data['DESCR_MARCA_VEICULO'].str[0:2] == "I/") | (all_data['DESCR_MARCA_VEICULO'].str[0:2] == "H/") | (all_data['DESCR_MARCA_VEICULO'].str[0:2] == "R/"),'DESCR_MARCA_VEICULO'] = all_data['DESCR_MARCA_VEICULO'].str[2:]
all_data.loc[all_data['DESCR_MARCA_VEICULO'].str[0:4] == "IMP/",'DESCR_MARCA_VEICULO'] = all_data['DESCR_MARCA_VEICULO'].str[4:]
all_data.loc[:,'DESCR_MARCA_VEICULO'] = all_data['DESCR_MARCA_VEICULO'].str.replace("NOVO ","")
all_data.loc[:,'DESCR_MARCA_VEICULO'] = all_data['DESCR_MARCA_VEICULO'].str.replace("NOVA ","")


In [12]:
all_data['MARCA'] = all_data['DESCR_MARCA_VEICULO'].str.split("/", expand=True)[0].str.split(" ", expand=True)[0]
all_data['MODELO'] = all_data['DESCR_MARCA_VEICULO'].str.split("/", expand=True)[1].str.split(" ", expand=True)[0]
all_data.loc[all_data.MODELO == 'VW','MODELO'] = all_data['DESCR_MARCA_VEICULO'].str.split("/", expand=True)[1].str.split(" ", expand=True)[1]


In [13]:
all_data.loc[all_data['MARCA'] == "CHEV",'MARCA'] = "CHEVROLET"
all_data.loc[all_data['MARCA'] == "GM",'MARCA'] = "CHEVROLET"
all_data.loc[(all_data['MARCA'] == "VOLKS") | (all_data['MARCA'] == "VOLKSWAGEN") | (all_data['MARCA'] == "VOLKSWAGEM"),'MARCA'] = "VW"
all_data.loc[(all_data['MARCA'] == "M.B.") | (all_data['MARCA'] == "MBENZ") | (all_data['MARCA'] == "MERCEDES"),'MARCA'] = "M.BENZ"
all_data.loc[all_data['MARCA'] == "MMC",'MARCA'] = "MITSUBISHI"
all_data.loc[all_data['MARCA'] == "IVECOFIAT",'MARCA'] = "IVECO"
all_data.loc[(all_data['MARCA'] == "LR") | (all_data['MARCA'] == "LROVER"),'MARCA'] = "LAND ROVER"


In [14]:
def ajuste_string(string):
        string = string.replace('Avenida ', 'Av. ').replace('Av ', 'Av. ').replace('Rua ', 'R. ').replace('R ', 'R. ')                     .replace('R ', 'R. ').replace(',', '.').replace('Praça ', 'Pr. ').replace('Pr ', 'Pr. ')                     .replace('Pública', 'pública').replace('Jardim', 'Jd.').replace('Jd ', 'Jd. ')
        return string

all_data.DESCRICAOLOCAL = all_data.astype(str).DESCRICAOLOCAL.apply(str)
all_data.LOGRADOURO = all_data.astype(str).LOGRADOURO.apply(str)
all_data.LATITUDE = all_data.LATITUDE.str.replace(',','.').astype(float)
all_data.LONGITUDE = all_data.LONGITUDE.str.replace(',','.').astype(float)

In [15]:
def ajusteData(df, col = 'DATAOCORRENCIA'):
    df[col] = pd.to_datetime(df[col], format='%d/%m/%Y', errors='coerce')

    indexs = []

    for row in df[col].items():
        if row[1] < pd.datetime(2010, 1, 1):
            indexs.append(row[0])

    df.drop(index=indexs, axis=1, inplace=True)

    df['DIA_SEMANA'] = df[col].dt.weekday
    df['MES_ANO'] = df[col].dt.strftime('%Y-%m')
    df['MES'] = df[col].dt.month

    df.drop(index=df[df['MES_ANO'] == 'NaT'].index, inplace=True, axis=0)

    return df

all_data = ajusteData(all_data)